In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import re
import pandas as pd
from datetime import datetime, timedelta
import logging
from google.colab import files
from tenacity import retry, wait_exponential, stop_after_attempt, retry_if_exception_type
import concurrent.futures
import threading
import glob
from tqdm import tqdm

# -----------------------------
# CONFIG
# -----------------------------
HEADERS = {"User-Agent": "Mozilla/5.0"}
FIRST_PAGE_URL = "https://newsday.co.tt/category/news"
BASE_PAGE_URL = "https://newsday.co.tt/category/news/page/"
LOG_DIR = "logs"
BATCH_DAYS = 1
MAX_WORKERS = 10  # threads per page
NUM_THREADS = 4   # threads for splitting the date range
START_DATE = datetime(2025, 7, 18).date()
END_DATE = datetime.now().date() - timedelta(days=1)  # yesterday
MAX_PAGE = 150  # approximate last page for your date range

os.makedirs(LOG_DIR, exist_ok=True)
logging.basicConfig(
    filename='scraper.log',
    filemode='a',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

def log_print(msg):
    print(msg)
    logging.info(msg)

# -----------------------------
# HELPER FUNCTIONS
# -----------------------------
def extract_date_from_url(url):
    match = re.search(r'/(\d{4})/(\d{2})/(\d{2})/', url)
    if match:
        return datetime.strptime(f"{match.group(1)}-{match.group(2)}-{match.group(3)}", "%Y-%m-%d").date()
    return None

@retry(wait=wait_exponential(multiplier=1, min=2, max=10),
       stop=stop_after_attempt(3),
       retry=retry_if_exception_type(Exception),
       reraise=True)
def scrape_article(url):
    res = requests.get(url, headers=HEADERS, timeout=10)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "html.parser")
    article = soup.find("article", class_="article-content")
    if not article:
        return None
    title = article.find("h1").text.strip() if article.find("h1") else "N/A"
    author = article.find("address", class_="article-author").text.strip() if article.find("address", class_="article-author") else "N/A"
    category = article.find("a", class_="article-category").text.strip() if article.find("a", class_="article-category") else "N/A"
    time_tag = article.find("time", class_="date-time")
    date_iso = time_tag["datetime"] if time_tag and "datetime" in time_tag.attrs else "N/A"
    body = "\n".join([p.get_text(strip=True) for p in article.find_all("p")])
    return {"url": url, "title": title, "author": author, "category": category, "date_iso": date_iso, "body": body}

# -----------------------------
# SCRAPE DATE RANGE
# -----------------------------
def scrape_date_range(start_date, end_date, save_path):
    articles_data = []

    # reverse pages: newest → oldest
    for page in range(MAX_PAGE, 0, -1):
        list_url = FIRST_PAGE_URL if page == 1 else f"{BASE_PAGE_URL}{page}/"
        log_print(f"Page {page}: {list_url}")

        try:
            res = requests.get(list_url, headers=HEADERS, timeout=10)
            res.raise_for_status()
            soup = BeautifulSoup(res.text, "html.parser")
            columns = soup.find_all("div", class_="column")
            if not columns:
                log_print("No articles found. End of archive?")
                break

            article_urls = []
            done = False
            for col in columns:
                article_block = col.find("article", class_="article-block")
                if not article_block:
                    continue
                a_tag = article_block.find("a", href=True)
                if not a_tag:
                    continue
                url = a_tag["href"]
                full_url = url if url.startswith("http") else "https://newsday.co.tt" + url
                date_obj = extract_date_from_url(full_url)
                if not date_obj:
                    continue
                if date_obj > end_date:
                    continue
                if date_obj < start_date:
                    done = True
                    break
                article_urls.append((full_url, date_obj))

            if not article_urls:
                continue

            # multithread per page
            with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
                future_to_url = {executor.submit(scrape_article, url): (url, date_obj) for url, date_obj in article_urls}
                for future in concurrent.futures.as_completed(future_to_url):
                    url, date_obj = future_to_url[future]
                    try:
                        data = future.result()
                        if data:
                            data["date_from_url"] = date_obj.isoformat()
                            articles_data.append(data)
                    except Exception as e:
                        log_print(f"Error scraping {url}: {e}")

            if done:
                break

        except Exception as e:
            log_print(f"Page error: {e}")
            continue

    if articles_data:
        df = pd.DataFrame(articles_data)
        df.to_parquet(save_path, index=False)
        files.download(save_path)
        log_print(f"Saved batch to {save_path}")
    else:
        log_print("No articles found in this batch.")

# -----------------------------
# MULTITHREADING ACROSS DAYS
# -----------------------------
def run_thread(thread_id, start, end):
    folder = f"data_batches_thread_{thread_id}"
    os.makedirs(folder, exist_ok=True)
    log_print(f"[Thread {thread_id}] Scraping {start} → {end}")
    current = start
    while current <= end:
        batch_end = min(current + timedelta(days=BATCH_DAYS-1), end)
        file_name = f"{folder}/newsday_{current}_to_{batch_end}.parquet"
        if not os.path.exists(file_name):
            scrape_date_range(current, batch_end, file_name)
        else:
            log_print(f"Batch exists, skipping: {file_name}")
        current = batch_end + timedelta(days=1)
    log_print(f"[Thread {thread_id}] Finished scraping.")

def merge_all_threads(output_file="newsday_all_data.parquet"):
    all_files = sorted(glob.glob("data_batches_thread_*/*.parquet"))
    if not all_files:
        log_print("⚠️ No parquet files found to merge.")
        return pd.DataFrame()
    dfs = [pd.read_parquet(f) for f in all_files]
    combined = pd.concat(dfs, ignore_index=True)
    combined.to_parquet(output_file, index=False)
    files.download(output_file)
    log_print(f"Merged {len(all_files)} files into {output_file}")
    return combined

# -----------------------------
# MAIN
# -----------------------------
if __name__ == "__main__":
    total_days = (END_DATE - START_DATE).days + 1
    chunk_size = total_days // NUM_THREADS

    threads = []
    for i in range(NUM_THREADS):
        start_chunk = START_DATE + timedelta(days=i*chunk_size)
        end_chunk = START_DATE + timedelta(days=(i+1)*chunk_size-1) if i < NUM_THREADS-1 else END_DATE
        t = threading.Thread(target=run_thread, args=(i+1, start_chunk, end_chunk))
        threads.append(t)
        t.start()

    for t in threads:
        t.join()

    # Merge all threads
    combined_df = merge_all_threads()
    print("Scraping complete. Total articles:", len(combined_df))


[Thread 1] Scraping 2025-07-18 → 2025-08-18
Page 150: https://newsday.co.tt/category/news/page/150/
[Thread 2] Scraping 2025-08-19 → 2025-09-19
Page 150: https://newsday.co.tt/category/news/page/150/
[Thread 4] Scraping 2025-10-22 → 2025-11-23
Page 150: https://newsday.co.tt/category/news/page/150/
[Thread 3] Scraping 2025-09-20 → 2025-10-21
Page 150: https://newsday.co.tt/category/news/page/150/
Page 149: https://newsday.co.tt/category/news/page/149/
Page 149: https://newsday.co.tt/category/news/page/149/
Page 149: https://newsday.co.tt/category/news/page/149/
Page 148: https://newsday.co.tt/category/news/page/148/
Page 148: https://newsday.co.tt/category/news/page/148/
Page 147: https://newsday.co.tt/category/news/page/147/
Page 149: https://newsday.co.tt/category/news/page/149/
Page 148: https://newsday.co.tt/category/news/page/148/
Page 148: https://newsday.co.tt/category/news/page/148/
Page 147: https://newsday.co.tt/category/news/page/147/
Page 147: https://newsday.co.tt/category

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saved batch to data_batches_thread_1/newsday_2025-07-18_to_2025-07-18.parquet
Page 150: https://newsday.co.tt/category/news/page/150/
Page 111: https://newsday.co.tt/category/news/page/111/
Page 149: https://newsday.co.tt/category/news/page/149/
Page 112: https://newsday.co.tt/category/news/page/112/
Page 148: https://newsday.co.tt/category/news/page/148/
Page 111: https://newsday.co.tt/category/news/page/111/
Page 147: https://newsday.co.tt/category/news/page/147/
Page 110: https://newsday.co.tt/category/news/page/110/
Page 146: https://newsday.co.tt/category/news/page/146/
Page 145: https://newsday.co.tt/category/news/page/145/
Page 144: https://newsday.co.tt/category/news/page/144/
Page 111: https://newsday.co.tt/category/news/page/111/
Page 143: https://newsday.co.tt/category/news/page/143/
Page 110: https://newsday.co.tt/category/news/page/110/
Page 142: https://newsday.co.tt/category/news/page/142/
Page 109: https://newsday.co.tt/category/news/page/109/
Page 141: https://newsday.

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

Saved batch to data_batches_thread_4/newsday_2025-11-23_to_2025-11-23.parquet
[Thread 4] Finished scraping.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Merged 129 files into newsday_all_data.parquet
Scraping complete. Total articles: 1239


# Exploring downloaded news parquet file

In [ ]:
import pandas as pd

news = pd.read_parquet('/content/newsday_all_data.parquet')
display(news.head())
display(news.tail())

,url,title,author,category,date_iso,body,date_from_url
0,https://newsday.co.tt/2025/07/18/state-of-emer...,State of Emergency in effect after murder plot...,Darren Bahaw,News,2025-07-18,A state of emergency went into effect on July ...,2025-07-18
1,https://newsday.co.tt/2025/07/18/roger-alexand...,Roger Alexander: PNM did not care about police...,Sean Douglas,News,2025-07-18,MINISTER of Homeland Security Roger Alexander ...,2025-07-18
2,https://newsday.co.tt/2025/07/18/security-offi...,Security officials tackle police trust crisis,Joey Bartlett,News,2025-07-18,ON THE HEELS of a July 13 investigative report...,2025-07-18
3,https://newsday.co.tt/2025/07/18/rehab-works-o...,Rehab works on UWI Debe campus to begin on Friday,Rishard Khan,News,2025-07-18,Rehabilitation works on the University of the ...,2025-07-18
4,https://newsday.co.tt/2025/07/18/fyzabad-man-f...,Fyzabad man freed of causing death by dangerou...,Laurel V Williams,News,2025-07-18,"A man from Fyzabad, who was charged with causi...",2025-07-18


,url,title,author,category,date_iso,body,date_from_url
1234,https://newsday.co.tt/2025/11/22/lee-48-hdc-ma...,Lee: 48 HDC maintenance contractors cut,Andrew Gioannetti,News,2025-11-22,ONLY 48 Housing Development Corporation (HDC) ...,2025-11-22
1235,https://newsday.co.tt/2025/11/22/babwah-trinid...,Babwah: Trinidad and Tobago could be 'dumping ...,Laurel V Williams,News,2025-11-22,"Visham Babwah, president of the TT Automotive ...",2025-11-22
1236,https://newsday.co.tt/2025/11/22/government-pa...,Government passes crypto-currency bill after '...,Gregory Mc Burnie,News,2025-11-22,Debate on the government’s proposed bill to re...,2025-11-22
1237,https://newsday.co.tt/2025/11/23/cops-move-to-...,Cops move to crush illegal quarries,Andrew Gioannetti,News,2025-11-23,The Multi-Agency Task Force is moving to crush...,2025-11-23
1238,https://newsday.co.tt/2025/11/23/gadsby-dolly-...,Gadsby-Dolly: PM must apologise for threat to ...,Clint Chan Tack,News,2025-11-23,St Ann's East MP Dr Nyan Gadsby-Dolly has cond...,2025-11-23


In [ ]:
print(news.info())
news['date_iso'] = pd.to_datetime(news['date_iso'])
news['date_from_url'] = pd.to_datetime(news['date_from_url'])
print(news.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1239 entries, 0 to 1238
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   url            1239 non-null   object
 1   title          1239 non-null   object
 2   author         1239 non-null   object
 3   category       1239 non-null   object
 4   date_iso       1239 non-null   object
 5   body           1239 non-null   object
 6   date_from_url  1239 non-null   object
dtypes: object(7)
memory usage: 67.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1239 entries, 0 to 1238
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   url            1239 non-null   object        
 1   title          1239 non-null   object        
 2   author         1239 non-null   object        
 3   category       1239 non-null   object        
 4   date_iso       1239 non-null   datetime64[ns

In [ ]:
news = news.sort_values(by='date_from_url').reset_index(drop=True)

display(news.tail(20))


,url,title,author,category,date_iso,body,date_from_url
1219,https://newsday.co.tt/2025/11/21/moonilal-open...,Moonilal: Open door to Nutrien and others,Clint Chan Tack,News,2025-11-21,ENERGY Minister Dr Roodal Moonilal says govern...,2025-11-21
1220,https://newsday.co.tt/2025/11/21/cpo-tells-psa...,CPO tells PSA head: Trinidad and Tobago's econ...,Clint Chan Tack,News,2025-11-21,CHIEF Personnel Officer (CPO) Commander Dr Dar...,2025-11-21
1221,https://newsday.co.tt/2025/11/21/psa-receives-...,PSA receives 10% offer by CPO,Stephon Nicholas,News,2025-11-21,PUBLIC Services Association president Felisha ...,2025-11-21
1222,https://newsday.co.tt/2025/11/21/khan-urges-ut...,Khan urges UTT graduates to help improve society,Dillon De Shong,News,2025-11-21,ATTORNEY Israel Khan SC is urging graduates of...,2025-11-21
1223,https://newsday.co.tt/2025/11/22/babwah-trinid...,Babwah: Trinidad and Tobago could be 'dumping ...,Laurel V Williams,News,2025-11-22,"Visham Babwah, president of the TT Automotive ...",2025-11-22
1224,https://newsday.co.tt/2025/11/22/senator-repor...,Senator reports Facebook death threats to cops,Newsday Reporter,News,2025-11-22,OPPOSITION Senator Dr Amery Browne has reporte...,2025-11-22
1225,https://newsday.co.tt/2025/11/22/howai-energy-...,"Howai: Energy diversification, renewable energ...",Ryan Hamilton-Davis,News,2025-11-22,Central Bank Governor Larry Howai highlighted ...,2025-11-22
1226,https://newsday.co.tt/2025/11/22/paint-powder-...,"Paint, powder and vibes as hundreds celebrate ...",Newsday Reporter,News,2025-11-22,REVELLERS took to the streets of San Fernando ...,2025-11-22
1227,https://newsday.co.tt/2025/11/22/prison-office...,"Prison officer, inmates charged with possessio...",Newsday Reporter,News,2025-11-22,Police have charged ten inmates and a prison o...,2025-11-22
1228,https://newsday.co.tt/2025/11/22/mp-told-in-pa...,MP told in Parliament: 'Go and take yuh lil man',Gregory Mc Burnie,News,2025-11-22,A parliamentarian was heard in a hot-mic momen...,2025-11-22


In [ ]:
import pandas as pd

# Convert 'date_from_url' to datetime if not already
news['date_from_url'] = pd.to_datetime(news['date_from_url'])

# Remove rows with the date 23-11-2025
news = news[news['date_from_url'] != pd.Timestamp('2025-11-23')].reset_index(drop=True)

# Sort by date_from_url
news = news.sort_values(by='date_from_url').reset_index(drop=True)

display(news.tail(20))

# Define start and end dates for filename
start_date = news['date_from_url'].min().strftime('%Y-%m-%d')
end_date = news['date_from_url'].max().strftime('%Y-%m-%d')

# Save as Parquet
filename = f"newsday_articles_{start_date}_to_{end_date}.parquet"
news.to_parquet(filename, index=False)
print(f"Saved file as: {filename}")


,url,title,author,category,date_iso,body,date_from_url
1217,https://newsday.co.tt/2025/11/21/trade-ministe...,Trade Minister: Venezuela a potential gold min...,Ryan Hamilton-Davis,News,2025-11-21,DESPITE mounting diplomatic tensions between t...,2025-11-21
1218,https://newsday.co.tt/2025/11/21/lawyer-tells-...,Lawyer tells DPP: Disregard Cepep lawsuit file...,Jada Loutoo,News,2025-11-21,ATTORNEYS for ex-Cepep contractor Eastman Ente...,2025-11-21
1219,https://newsday.co.tt/2025/11/21/sturge-mum-on...,Sturge mum on whether government allowing US w...,Gregory Mc Burnie,News,2025-11-21,DEFENCE Minister Wayne Sturge has cited nation...,2025-11-21
1220,https://newsday.co.tt/2025/11/21/gary-aboud-cl...,Gary Aboud claims US visa cancelled over criti...,Stephon Nicholas,News,2025-11-21,FISHERMEN & Friends of the Sea (FFOS) corporat...,2025-11-21
1221,https://newsday.co.tt/2025/11/21/permanent-sec...,Permanent Secretary robbed by 4 men in Cascade,Joey Bartlett,News,2025-11-21,PERMANENT Secretary in the Ministry of Trade a...,2025-11-21
1222,https://newsday.co.tt/2025/11/21/diego-woman-d...,Diego woman detained for alleged online posts ...,Jada Loutoo,News,2025-11-21,HOMELAND Security Minister Roger Alexander has...,2025-11-21
1223,https://newsday.co.tt/2025/11/22/ema-seeks-pub...,EMA seeks public's views on proposed Rocky Poi...,Newsday,News,2025-11-22,Alva Viarruel\nThe start of construction of a ...,2025-11-22
1224,https://newsday.co.tt/2025/11/22/trade-ministe...,Trade minister tells Trinbago Express Shipping...,Dillon De Shong,News,2025-11-22,Parliamentary Secretary in the Ministry of Tra...,2025-11-22
1225,https://newsday.co.tt/2025/11/22/lee-48-hdc-ma...,Lee: 48 HDC maintenance contractors cut,Andrew Gioannetti,News,2025-11-22,ONLY 48 Housing Development Corporation (HDC) ...,2025-11-22
1226,https://newsday.co.tt/2025/11/22/ramadhar-lead...,Ramadhar leads legal probe of gas tank explosi...,Rishard Khan,News,2025-11-22,Attorneys probing Ria George's death vow that ...,2025-11-22


Saved file as: newsday_articles_2025-07-18_to_2025-11-22.parquet
